In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("Data/merged_tk2025_dataset.csv")
df.columns

Index(['gemeente_naam', 'population', 'num_households', 'avg_income_household',
       'median_income_household', 'pct_migration_background', 'votes_total',
       '50PLUS_share_pct', 'BBB_share_pct', 'CDA_share_pct', 'D66_share_pct',
       'DENK_share_pct', 'JA21_share_pct', 'VVD_share_pct', 'VOLT_share_pct',
       'econ_CENTERLEFT_share_pct', 'econ_RIGHT_share_pct',
       'econ_LEFT_share_pct', 'econ_CENTERRIGHT_share_pct',
       'mig_PROGRESSIVE_share_pct', 'mig_EXTREME_CONSERVATIVE_share_pct',
       'mig_CONSERVATIVE_share_pct', 'mig_EXTREME_PROGRESSIVE_share_pct'],
      dtype='object')